In [17]:
import pandas as pd

In [23]:
harry = pd.read_csv("sbin.csv")
df = pd.DataFrame(harry)
df

Timestamp    Open    High     Low   Close  Volume   tr0  \
0     2023-03-15T09:15:00+0530  529.80  531.85  529.60  530.90  253865  2.25   
1     2023-03-15T09:16:00+0530  530.95  531.30  529.80  530.50  122662  1.50   
2     2023-03-15T09:17:00+0530  530.20  530.50  529.05  529.60  117946  1.45   
3     2023-03-15T09:18:00+0530  529.70  529.80  529.10  529.65   84080  0.70   
4     2023-03-15T09:19:00+0530  529.65  530.00  529.05  529.20   87324  0.95   
...                        ...     ...     ...     ...     ...     ...   ...   
1121  2023-03-17T15:26:00+0530  531.45  531.60  531.35  531.45  349090  0.25   
1122  2023-03-17T15:27:00+0530  531.35  531.50  531.05  531.35  177466  0.45   
1123  2023-03-17T15:28:00+0530  531.05  531.30  530.80  531.00   73802  0.50   
1124  2023-03-17T15:29:00+0530  531.00  531.40  529.85  531.00   85647  1.55   
1125  2023-03-17T15:30:00+0530  531.00  531.00  531.00  531.00     580  0.00   

       tr1   tr2    TR       ATR     BUB     BLB     FUB     FLB      ST  \
0      NaN   NaN  2.25  0.000000  530.72  530.72    0.00    0.00    0.00   
1     0.40  1.10  1.50  0.107143  530.76  530.34  530.76  530.34  530.76   
2     0.00  1.45  1.45  0.203061  530.18  529.37  530.18  530.34  530.18   
3     0.20  0.50  0.70  0.238557  529.93  528.97  529.93  528.97  529.93   
4     0.35  0.60  0.95  0.289374  530.10  528.95  529.93  528.97  529.93   
...    ...   ...   ...       ...     ...     ...     ...     ...     ...   
1121  0.20  0.05  0.25  0.658139  532.79  530.16  531.46  530.16  530.16   
1122  0.05  0.40  0.45  0.643272  532.56  529.99  531.46  530.16  530.16   
1123  0.05  0.55  0.55  0.636610  532.32  529.78  531.46  530.16  530.16   
1124  0.40  1.15  1.55  0.701852  532.03  529.22  531.46  530.16  530.16   
1125  0.00  0.00  0.00  0.651720  532.30  529.70  531.46  530.16  530.16   

     ST_BUY_SELL  
0            NaN  
1           SELL  
2           SELL  
3           SELL  
4           SELL  
...          ...  
1121         BUY  
1122         BUY  
1123         BUY  
1124         BUY  
1125         BUY  

[1126 rows x 17 columns]

In [32]:


data=df
data['tr0'] = abs(data["High"] - data["Low"])
data['tr1'] = abs(data["High"] - data["Close"].shift(1))
data['tr2'] = abs(data["Low"]- data["Close"].shift(1))
data["TR"] = round(data[['tr0', 'tr1', 'tr2']].max(axis=1),2)
data["ATR"]=0.00
data['BUB']=0.00
data["BLB"]=0.00
data["FUB"]=0.00
data["FLB"]=0.00
data["ST"]=0.00

# Calculating ATR 
for i, row in data.iterrows():
    if i == 0:
        data.loc[i,'ATR'] = 0.00#data['ATR'].iat[0]
    else:
        data.loc[i,'ATR'] = ((data.loc[i-1,'ATR'] * 13)+data.loc[i,'TR'])/14

data['BUB'] = round(((data["High"] + data["Low"]) / 2) + (2 * data["ATR"]),2)
data['BLB'] = round(((data["High"] + data["Low"]) / 2) - (2 * data["ATR"]),2)


# FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
#                     THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)


for i, row in data.iterrows():
    if i==0:
        data.loc[i,"FUB"]=0.00
    else:
        if (data.loc[i,"BUB"]<data.loc[i-1,"FUB"])|(data.loc[i-1,"Close"]>data.loc[i-1,"FUB"]):
            data.loc[i,"FUB"]=data.loc[i,"BUB"]
        else:
            data.loc[i,"FUB"]=data.loc[i-1,"FUB"]

# FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
#                     THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)

for i, row in data.iterrows():
    if i==0:
        data.loc[i,"FLB"]=0.00
    else:
        if (data.loc[i,"BLB"]>data.loc[i-1,"FLB"])|(data.loc[i-1,"Close"]<data.loc[i-1,"FLB"]):
            data.loc[i,"FLB"]=data.loc[i,"BLB"]
        else:
            data.loc[i,"FLB"]=data.loc[i-1,"FLB"]



# SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
#                 Current FINAL UPPERBAND
#             ELSE
#                 IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
#                     Current FINAL LOWERBAND
#                 ELSE
#                     IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
#                         Current FINAL LOWERBAND
#                     ELSE
#                         IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
#                             Current FINAL UPPERBAND


for i, row in data.iterrows():
    if i==0:
        data.loc[i,"ST"]=0.00
    elif (data.loc[i-1,"ST"]==data.loc[i-1,"FUB"]) & (data.loc[i,"Close"]<=data.loc[i,"FUB"]):
        data.loc[i,"ST"]=data.loc[i,"FUB"]
    elif (data.loc[i-1,"ST"]==data.loc[i-1,"FUB"])&(data.loc[i,"Close"]>data.loc[i,"FUB"]):
        data.loc[i,"ST"]=data.loc[i,"FLB"]
    elif (data.loc[i-1,"ST"]==data.loc[i-1,"FLB"])&(data.loc[i,"Close"]>=data.loc[i,"FLB"]):
        data.loc[i,"ST"]=data.loc[i,"FLB"]
    elif (data.loc[i-1,"ST"]==data.loc[i-1,"FLB"])&(data.loc[i,"Close"]<data.loc[i,"FLB"]):
        data.loc[i,"ST"]=data.loc[i,"FUB"]

# Buy Sell Indicator
for i, row in data.iterrows():
    if i==0:
        data["ST-BUY_SELL"]="NA"
    elif (data.loc[i,"ST"]<data.loc[i,"Close"]) :
        data.loc[i,"ST-BUY_SELL"]="BUY"
    else:
        data.loc[i,"ST-BUY_SELL"]="SELL"
 # Buy Sell Indicator
for i, row in data.iterrows():
    if i==0:
        data["STX"]="NA"
    elif (data.loc[i,"ST"]<data.loc[i,"Close"]) :
        data.loc[i,"STX"]="green"
    else:
        data.loc[i,"STX"]="red"       

In [33]:
df

Timestamp    Open    High     Low   Close  Volume   tr0  \
0     2023-03-15T09:15:00+0530  529.80  531.85  529.60  530.90  253865  2.25   
1     2023-03-15T09:16:00+0530  530.95  531.30  529.80  530.50  122662  1.50   
2     2023-03-15T09:17:00+0530  530.20  530.50  529.05  529.60  117946  1.45   
3     2023-03-15T09:18:00+0530  529.70  529.80  529.10  529.65   84080  0.70   
4     2023-03-15T09:19:00+0530  529.65  530.00  529.05  529.20   87324  0.95   
...                        ...     ...     ...     ...     ...     ...   ...   
1121  2023-03-17T15:26:00+0530  531.45  531.60  531.35  531.45  349090  0.25   
1122  2023-03-17T15:27:00+0530  531.35  531.50  531.05  531.35  177466  0.45   
1123  2023-03-17T15:28:00+0530  531.05  531.30  530.80  531.00   73802  0.50   
1124  2023-03-17T15:29:00+0530  531.00  531.40  529.85  531.00   85647  1.55   
1125  2023-03-17T15:30:00+0530  531.00  531.00  531.00  531.00     580  0.00   

       tr1   tr2    TR       ATR     BUB     BLB     FUB     FLB      ST  \
0      NaN   NaN  2.25  0.000000  530.72  530.72    0.00    0.00    0.00   
1     0.40  1.10  1.50  0.107143  530.76  530.34  530.76  530.34  530.76   
2     0.00  1.45  1.45  0.203061  530.18  529.37  530.18  530.34  530.18   
3     0.20  0.50  0.70  0.238557  529.93  528.97  529.93  528.97  529.93   
4     0.35  0.60  0.95  0.289374  530.10  528.95  529.93  528.97  529.93   
...    ...   ...   ...       ...     ...     ...     ...     ...     ...   
1121  0.20  0.05  0.25  0.658139  532.79  530.16  531.46  530.16  530.16   
1122  0.05  0.40  0.45  0.643272  532.56  529.99  531.46  530.16  530.16   
1123  0.05  0.55  0.55  0.636610  532.32  529.78  531.46  530.16  530.16   
1124  0.40  1.15  1.55  0.701852  532.03  529.22  531.46  530.16  530.16   
1125  0.00  0.00  0.00  0.651720  532.30  529.70  531.46  530.16  530.16   

     ST_BUY_SELL ST-BUY_SELL    STX  
0            NaN          NA     NA  
1           SELL        SELL    red  
2           SELL        SELL    red  
3           SELL        SELL    red  
4           SELL        SELL    red  
...          ...         ...    ...  
1121         BUY         BUY  green  
1122         BUY         BUY  green  
1123         BUY         BUY  green  
1124         BUY         BUY  green  
1125         BUY         BUY  green  

[1126 rows x 19 columns]